In [ ]:
# Importar las bibliotecas necesarias
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count,desc,sum,to_date,date_format,concat_ws,format_string,to_timestamp,year,when,lit,month#,reduce
from functools import reduce
from pyspark.sql import Window
import pandas as pd

# Crear una SparkSession
spark = SparkSession.builder.appName("AzureSQL").getOrCreate()

## Credenciales BD
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Tuboleta/Credenciales.txt"
keys = pd.read_csv(inpath, sep = ',')
display(keys)


# Creo variables para cada fila del DataFrame que contiene las credenciales de la bd
for index, row in keys.iterrows():
    variable_name = row['key']
    variable_value = row['value']
    globals()[variable_name] = variable_value

# Configurar las propiedades de la conexión
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:1433;database={jdbc_database_datamart}"
jdbc_properties = {
    "user": user,
    "password": password,
    "driver": driver
}

# Conexion al Blob
spark.conf.set(clave_blob, access_key_blob)

In [ ]:
# CONFIGURATION
inpath = 'abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Configuration/Ds_S_Configuration.parquet'
config = spark.read.parquet(inpath)

# FOLLWUP
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/silver/Follwup/Ds_S_Follwup.parquet"
follwup = spark.read.parquet(inpath)
display(follwup.limit(5))

Key_evento_,PRODUCT_FAMILY,Nuevo_aforo
101403083159|101403135400|GENERAL,Evento,200000.0
101412774479|101412662473|GENERAL,Evento,200000.0
101412774479|101412662472|GENERAL,Evento,200000.0
101403083148|101403135388|GENERAL,Evento,200000.0
101403083157|101403135394|GENERAL,Evento,200000.0


In [ ]:
# USO DE UN LIKE EN SIMULTANEO PARA FILTRAR VARIOS EVENTOS 
eventos = ['10229674770319', '10229674777288', '10229674777289', '10229674777287']

config = config.filter(col('T_PRODUCT_ID').isin(eventos))

# Crear la condición de filtro usando like y or
#condiciones = [col("Key_evento_").like(f"%{valor}%") for valor in eventos] contains
condiciones = [col("Key_evento_").contains(valor) for valor in eventos] 

# Aplicar el filtro
follwup = follwup.filter(reduce(lambda x, y: x | y, condiciones))

#tmp = follwup.select('Key_evento_').distinct()
#display(tmp.limit(200))

In [ ]:
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Hot_Sale_Maestro/silver/Configuration/"
# 2024_final
join_df_coalesced = config.coalesce(1)
#join_df_coalesced = Ds_S3_Sales_2024.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# En la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_S_Configuration.parquet")

True

In [ ]:
# Guarda el DataFrame en formato Parquet en Azure Blob Storage
inpath = "abfss://storagebi@tbdwhstorage01.dfs.core.windows.net/Hot_Sale_Maestro/silver/Follwup/"
# 2024_final
join_df_coalesced = follwup.coalesce(1)
#join_df_coalesced = Ds_S3_Sales_2024.coalesce(1)
join_df_coalesced.write.mode("overwrite").parquet(inpath)

#cambio de nombre
files=dbutils.fs.ls(inpath)
output_file= [x for x in files if x.name.startswith("part-")]

# En la misma carpeta de origen
dbutils.fs.mv(output_file[0].path, f"{inpath}/Ds_S_follwup.parquet")

True